In [52]:
import json
import math
import fiona
import folium
import branca.colormap as cm
import requests
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Polygon
import urllib.request
from urllib import request
from shapely.geometry import shape, Point
import os
import threading
from rtree import index
import time

In [53]:
def idw(lat , lon , ref_point , ref_number):     
    total_out = 0
    total_dis = 0
    # optional normalize
    size = 0.01
    sort_list = []
    # sort distance
    for index,row in ref_point.iterrows():           # 使用.iterrows() 一列一列讀取資料
        ref_lon = row['Longitude']
        ref_lat = row['Latitude']
        distance = math.sqrt(((ref_lon - lon)/size) ** 2 + ((ref_lat - lat)/size) ** 2)
        sort_list.append([ref_lon,ref_lat,row['PM2.5'],distance])
    sort_list = sorted(sort_list,key=lambda l:l[3], reverse=False) # 根據距離近到遠排序
    count = 0
    # top ref_number point 找出距離最近的幾個測站，把1/distance累加
    for s in sort_list: # sort_list : [ref_lon,ref_lat,row['PM2.5'],distance] s[2] : pm2.5 , s[3]:distance
        if count == ref_number:
            break
        count += 1
        total_dis += 1 / s[3] #s[3] = distance 1/總距離 (1/distance 1 + 1/distance 2 + ..... 1/distance n)
    count = 0
    # idw : 
    # total out : idw預估的pm2.5 -> sigma(前n個測站的pm2.5 * ((1/distance) / 總距離))
    # 這個測站的pm2.5 * 這個測站多重要（權重）
    # 權重：(1/測站離預估點的距離) / (1/總距離) （總距離：前n個測站的距離倒數的和）
    for s in sort_list:
        if count == ref_number:
            break
        count += 1
        total_out += ((1 / s[3]) / total_dis) * s[2] #權重 * pm2.5
    return total_out # 預估的pm2.5


In [54]:
#========================================================================================================
#爬蟲 (環保署測站、時間、風力資訊、台中各區天氣) + idw

ses = requests.Session()
data1 = ses.get('http://taqm.epb.taichung.gov.tw/TQAMNEWAQITABLE.ASPX') #環保署16筆測站
data1.encoding = 'utf-8'
t = pd.read_html(data1.text)[0]
t.drop(t.iloc[:, 1:21], inplace=True, axis=0)
t.drop(t.iloc[:, 1:279], inplace=True, axis=1)
times = str(t[0])



In [55]:
#爬蟲 微型感測器

url = 'https://aqi.thu.edu.tw/echarts/getEPAIotData'
json_data = request.urlopen(url).read().decode("utf-8")
json_data = json.loads(json_data);
frame = pd.DataFrame.from_dict(json_data);

frame['areaname']=''
frame.index= range(1,len(frame) + 1)
frame['Id']=frame.index
frame.index= range(0,len(frame))
times_micro = str(frame['datetime'])


In [56]:
#找出位於台中的微型感測器
def get_areanamebythread(totaldata):
    totalnum=totaldata  #總執行次數
    Q=int(totalnum/10) #取商數
    R=totalnum%10      #取餘數
    
    for i in range(Q):
        threads = []
        for j in range(10):
            threads.append(threading.Thread(
                target=getareaname,#要執行函數
                args=(frame['longitude'][i*10+j],frame['latitude'][i*10+j],i*10+j)))#要執行函數的參數
            threads[j].start()
        for j in threads:
            j.join()
        print(round(float((i+1)*100/totalnum*10),2),'%')#顯示進度
    
    threads = []
    for i in range(R):
        threads.append(threading.Thread(
            target=getareaname,
            args=(frame['longitude'][Q*10+i],frame['latitude'][Q*10+i],Q*10+i)))
        threads[i].start()
    for j in threads:
        j.join()
    print("100%")#顯示進度

def search(x, y):  #尋找鄉鎮
    global shapes, townnames
    #idx = index.Index()
    #for town_id, shape in shapes.items():
    #idx.insert(town_id,shape.bounds)
    return next((townnames[town_id]  #如果鄉鎮區域包含傳入的經緯度就傳回townnames[town_id]
                 for town_id in shapes #idx.intersection((x,y)) #逐一尋找各鄉鎮
                 if shapes[town_id].contains(Point(x, y))), None)

area_append = pd.DataFrame(columns=['areaname'])
area_append['areaname'] = frame['datetime']

def getareaname(lng,lat,num):
   
    lng = float(lng)
    lat = float(lat)
    
    area_append['areaname'][num]=search(float(lng), float(lat))
    return 0

module_dir = os.path.dirname('/home/gh555657/123321/areasearch/')  #取得目前目錄
collection = fiona.open(os.path.join(module_dir,'TOWN_MOI_1070516.shp'))
shapes = {}
townnames = {}

for f in collection:
    town_id =f['properties']['TOWNCODE'] #鄉鎮代碼
    shapes[town_id] = shape(f['geometry'])  #鄉鎮界限經緯度
    townnames[town_id] = f['properties']['COUNTYNAME'] + f['properties']['TOWNNAME']#search函式傳回值
    
t1=time.time()
get_areanamebythread(frame.shape[0])
t2=time.time()
print('gevent-time:%s' % str(t2-t1))
frame['areaname']=area_append['areaname']


0.35 %
0.7 %
1.06 %
1.41 %
1.76 %
2.11 %
2.46 %
2.82 %
3.17 %
3.52 %
3.87 %
4.23 %
4.58 %
4.93 %
5.28 %
5.63 %
5.99 %
6.34 %
6.69 %
7.04 %
7.39 %
7.75 %
8.1 %
8.45 %
8.8 %
9.15 %
9.51 %
9.86 %
10.21 %
10.56 %
10.92 %
11.27 %
11.62 %
11.97 %
12.32 %
12.68 %
13.03 %
13.38 %
13.73 %
14.08 %
14.44 %
14.79 %
15.14 %
15.49 %
15.85 %
16.2 %
16.55 %
16.9 %
17.25 %
17.61 %
17.96 %
18.31 %
18.66 %
19.01 %
19.37 %
19.72 %
20.07 %
20.42 %
20.77 %
21.13 %
21.48 %
21.83 %
22.18 %
22.54 %
22.89 %
23.24 %
23.59 %
23.94 %
24.3 %
24.65 %
25.0 %
25.35 %
25.7 %
26.06 %
26.41 %
26.76 %
27.11 %
27.46 %
27.82 %
28.17 %
28.52 %
28.87 %
29.23 %
29.58 %
29.93 %
30.28 %
30.63 %
30.99 %
31.34 %
31.69 %
32.04 %
32.39 %
32.75 %
33.1 %
33.45 %
33.8 %
34.15 %
34.51 %
34.86 %
35.21 %
35.56 %
35.92 %
36.27 %
36.62 %
36.97 %
37.32 %
37.68 %
38.03 %
38.38 %
38.73 %
39.08 %
39.44 %
39.79 %
40.14 %
40.49 %
40.85 %
41.2 %
41.55 %
41.9 %
42.25 %
42.61 %
42.96 %
43.31 %
43.66 %
44.01 %
44.37 %
44.72 %
45.07 %
45.42 %
45.77 %


In [57]:
frame['pm25'].replace({"NA":np.nan}, inplace=True)
frame['pm25'].astype('float64')
frame['pm25'].replace({"-99":np.nan,"-99.0":np.nan}, inplace=True)
frame[frame['pm25']<0]=np.nan
frame.dropna(inplace=True)
frame

,datetime,device_id,latitude,longitude,pm25,areaname,Id
0,2019-12-21 02:03:26,1858224692,24.975056,121.330402,7.0,新北市鶯歌區,1.0
3,2019-12-21 02:47:10,1858569336,24.967298,121.339279,8.0,新北市鶯歌區,4.0
4,2019-12-21 02:48:11,1858763627,24.973724,121.332481,11.0,新北市鶯歌區,5.0
5,2019-12-21 02:49:42,1858852981,24.972902,121.342242,10.0,新北市鶯歌區,6.0
6,2019-12-20 14:36:39,1858981737,24.976159,121.334666,7.0,新北市鶯歌區,7.0
7,2019-12-20 08:04:43,1859886331,24.965728,121.332249,8.0,新北市鶯歌區,8.0
8,2019-12-21 02:50:18,1860067717,24.94805,121.341753,8.0,新北市鶯歌區,9.0
9,2019-12-21 02:42:36,1860271203,24.956835,121.360893,7.0,新北市鶯歌區,10.0
10,2019-12-21 02:49:55,1860464577,24.973736,121.328505,11.0,新北市鶯歌區,11.0
11,2019-12-21 02:45:45,1860726455,24.95694,121.358481,7.0,新北市鶯歌區,12.0


In [58]:
epa = frame.loc[frame["areaname"].str.contains('台中|臺中')]
epa=epa.drop(columns=['datetime','areaname']).reset_index(drop=True)
epa.columns=['SiteName','Latitude','Longitude','PM2.5','Id']
epa.index= range(1,len(epa) + 1)
epa['Id']=epa.index
epa.index= range(0,len(epa))
epaidw=epa
epa=epa[['SiteName','Id','PM2.5','Latitude','Longitude']]
epaidw=epaidw.drop(columns=['SiteName']).reset_index(drop=True)
epa

,SiteName,Id,PM2.5,Latitude,Longitude
0,6170369574,1,13.0,24.161848,120.6049575
1,6170479504,2,8.0,24.237501,120.517601
2,6170511673,3,6.0,24.241976,120.523674
3,6170644194,4,13.0,24.15854,120.6066032
4,6170777118,5,10.0,24.166212,120.5971678
5,6170852684,6,8.0,24.206862,120.5050963
6,6170988978,7,5.0,24.234808,120.5295139
7,6171080763,8,26.0,24.231787,120.5124511
8,6171173697,9,5.0,24.236878,120.5057026
9,6171278726,10,7.0,24.242703,120.5328221


In [59]:
epaidw

,Latitude,Longitude,PM2.5,Id
0,24.161848,120.6049575,13.0,1
1,24.237501,120.517601,8.0,2
2,24.241976,120.523674,6.0,3
3,24.15854,120.6066032,13.0,4
4,24.166212,120.5971678,10.0,5
5,24.206862,120.5050963,8.0,6
6,24.234808,120.5295139,5.0,7
7,24.231787,120.5124511,26.0,8
8,24.236878,120.5057026,5.0,9
9,24.242703,120.5328221,7.0,10


In [60]:
epaidw.columns=['Latitude','Longitude','PM2.5','Id']
epaidw=epaidw[['Id','Latitude','Longitude','PM2.5']]
epaidw

,Id,Latitude,Longitude,PM2.5
0,1,24.161848,120.6049575,13.0
1,2,24.237501,120.517601,8.0
2,3,24.241976,120.523674,6.0
3,4,24.15854,120.6066032,13.0
4,5,24.166212,120.5971678,10.0
5,6,24.206862,120.5050963,8.0
6,7,24.234808,120.5295139,5.0
7,8,24.231787,120.5124511,26.0
8,9,24.236878,120.5057026,5.0
9,10,24.242703,120.5328221,7.0


In [61]:
# print(times[21:36]) 時間
df1 = pd.read_html(data1.text)[1]

cols1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
df1 = df1.replace('─','0')
df1 = df1.drop(df1.columns[cols1],axis=1)
df1.rename(columns={ df1.columns[0]: "SiteName"}, inplace=True)
df1.rename(columns={ df1.columns[1]: "PM2.5"}, inplace=True)
df1['Latitude']=[24.1622,24.151958,24.099611,24.225628,
                 24.256586,24.139008,24.350426,24.139564,
                 24.05735,24.094264,24.307036,24.250388,
                 24.150919,24.182055,24.269233,24.20103]

df1['Longitude']=[120.616917,120.641092,120.677689,120.568794,
                  120.741711,120.597876,120.615358,120.715064,
                  120.697299,120.646629,120.714881,120.538839,
                  120.540877,120.60707,120.576421,120.498566]


In [62]:
df1.rename(columns={"測站名稱":"SiteName"}, inplace=True)
df1.rename(columns={ "細懸浮微粒(PM2.5)":"PM2.5"}, inplace=True)


df1

,SiteName,PM2.5,Latitude,Longitude
,SiteName,移動平均濃度(μg/m3),,
0,西屯測站(環保署),11,24.162200,120.616917
1,忠明測站(環保署),12,24.151958,120.641092
2,大里測站(環保署),12,24.099611,120.677689
3,沙鹿測站(環保署),9,24.225628,120.568794
4,豐原測站(環保署),9,24.256586,120.741711
5,文山測站,19,24.139008,120.597876
6,大甲測站,9,24.350426,120.615358
7,太平測站,14,24.139564,120.715064
8,霧峰測站,13,24.057350,120.697299


In [63]:
df1[['Latitude', 'Longitude', 'PM2.5']] = df1[['Latitude', 'Longitude','PM2.5']].astype(float)
#df1  顯示16筆測站
df1.to_csv("/home/gh555657/123321/df1.csv")
df1 = pd.read_csv("/home/gh555657/123321/df1.csv")



In [64]:
df1.drop([0],inplace=True) #把df1不要的第一row砍掉
df11=df1['SiteName']       #先測站把名稱存進df11
df11=df11.append(epa['SiteName'])#在把微型感測器測站名稱也併進df11

df1.drop(["SiteName"],axis=1,inplace=True)#測站名稱column砍掉
df1.rename(columns={ 'Unnamed: 0':'Id'}, inplace=True)#更改unname--->Id
#df1=df1[['Id','SiteName','PM2.5','Latitude','Longitude']] df1目前型態
#df1=df1.append(epaidw) #把微型感測器資料也併進來
df1=df1.astype('float64')#轉成float64
df11.index= range(0,len(df11))#index重排
df1.index=range(0,len(df1))

In [65]:
#taichung = gp.read_file("/home/hpc/taichungcity.geojson")           #台中邊界
taichungmap_1x1 = gp.read_file("/home/gh555657/123321/final.geojson")         #台中1*1網格
taichung_district = gp.read_file("/home/gh555657/123321/taichung_district.geojson")
#list1= [   1,    4,   14,   26,   44,   63,   82,  102,  122,  144,
#         168,  193,  221,  257,  304,  353,  403,  455,  510,  568,
#         627,  687,  750,  819,  892,  968, 1053, 1141, 1232, 1325,
#        1418, 1510, 1601, 1692, 1781, 1864, 1944, 2019, 2087, 2145,
#        2197, 2246, 2289, 2330, 2359, 2384, 2403, 2419, 2433, 2445 ]
#list2= [   3,   13,   25,   43,   62,   81,  101,  121,  143,  167,
#         192,  220,  256,  303,  352,  402,  454,  509,  567,  626,
#         686,  749,  818,  891,  967, 1052, 1140, 1231, 1324, 1417,
#        1509, 1600, 1691, 1780, 1863, 1943, 2018, 2086, 2144, 2196,
#        2245, 2288, 2329, 2358, 2383, 2402, 2418, 2432, 2444, 2449 ]
lon_max=taichungmap_1x1.bounds.maxx
lon_min=taichungmap_1x1.bounds.minx
lat_max=taichungmap_1x1.bounds.maxy
lat_min=taichungmap_1x1.bounds.miny



In [66]:
# df3 idw point
df3 = pd.DataFrame(columns=['Latitude', 'Longitude', 'PM2.5' , 'Id'])
site_name_count = 1
ref_point_number = 16      # edit here to change ref_number
for i,j,k,l in zip(lat_max,lat_min,lon_max,lon_min):
    site_name = str(site_name_count)
    site_name_count += 1
    
    df_append = pd.DataFrame([[(i+j)/2 , (k+l)/2 , 
                         idw((i+j)/2 , (k+l)/2 , df1 , ref_point_number),site_name]] 
                         ,columns=['Latitude', 'Longitude', 'PM2.5' , 'Id'])
    df3 = df3.append(df_append)        #合併

df3.to_csv("/home/gh555657/123321/testidw_original.csv")
all_point_data_epa = pd.read_csv("/home/gh555657/123321/testidw_original.csv")
lon=list(all_point_data_epa['Longitude'])
lat=list(all_point_data_epa['Latitude'])
#all_point_data_epa['Id']=0
Id=taichungmap_1x1['Id']
ans_Id=all_point_data_epa['Id']


In [67]:
def generalID(lon,lat,column_num,row_num):
    # 若在范围外的点，返回-1
    if lon <= LON1 or lon >= LON2 or lat <= LAT1 or lat >= LAT2:
        return -1
    # 把经度范围根据列数等分切割
    column = (LON2 - LON1)/column_num
    # 把纬度范围根据行数数等分切割
    row = (LAT2 - LAT1)/row_num
    # 二维矩阵坐标索引转换为一维ID，即： （列坐标区域（向下取整）+ 1） + （行坐标区域 * 列数）
    return int((lon-LON1)/column)+ 1 + int((lat-LAT1)/row) * column_num


In [68]:
ans_Id

0          1
1          2
2          3
3          4
4          5
5          6
6          7
7          8
8          9
9         10
10        11
11        12
12        13
13        14
14        15
15        16
16        17
17        18
18        19
19        20
20        21
21        22
22        23
23        24
24        25
25        26
26        27
27        28
28        29
29        30
        ... 
2419    2420
2420    2421
2421    2422
2422    2423
2423    2424
2424    2425
2425    2426
2426    2427
2427    2428
2428    2429
2429    2430
2430    2431
2431    2432
2432    2433
2433    2434
2434    2435
2435    2436
2436    2437
2437    2438
2438    2439
2439    2440
2440    2441
2441    2442
2442    2443
2443    2444
2444    2445
2445    2446
2446    2447
2447    2448
2448    2449
Name: Id, Length: 2449, dtype: int64

In [69]:
# fit id
#for i in range(2449):
#    ans_Id[i] = i+1
# for d in (range(100)):      #測試資料筆數
#      for m in (range(50)):  #找是在哪列
#             if lon_min[list1[m]-1]<lon[d] and lon[d]<lon_max[list2[m]-1] and lat_min[list1[m]-1]<lat[d] and lat[d]<lat_max[list2[m]-1]:
#                 for n in range(list1[m],list2[m]+1): #找是在該列的哪個
#                     if lon_min[n-1]<lon[d] and lon[d]<lon_max[n-1] and lat_min[n-1]<lat[d] and lat[d]<lat_max[n-1]:
#                         ans_Id[d]=Id[n]-1

#taichung_district

In [70]:
taichungmap_1x1 = taichungmap_1x1.merge(all_point_data_epa, on='Id')
taichungmap_1x1['PM2.5']=taichungmap_1x1['PM2.5'].round()



In [71]:
df1['SiteName']=df11
df1=df1[['Id','SiteName','PM2.5','Latitude','Longitude']]


In [72]:
# =============================================================================================================
# folium

variable = 'PM2.5'
colorList = [
    '#98fb98', '#51ff51', '#00ff00', '#1ce11c', '#32cd32', '#ffff00',
    '#ffee00', '#ffd13f', '#ffc700', '#ffbf4a', '#ffa500', '#ff6347',
    '#ff5047', '#ff4c2c', '#ff0000', '#d32c4a', '#ba55d3'
]
map_color = cm.StepColormap(
    colorList,
    index=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80],
    vmin=0,
    vmax=150,
    caption='PM2.5')

fmap = folium.Map(location=[24.2, 120.9], zoom_start=10.5)
# fmap.choropleth(
#                geo_data=taichungmap_1x1,
#                name='pm2.5',
#                columns=['Id', 'PM2.5'],
#                key_on='feature.properties.Id',
#                data=all_point_data_epa,
#               #threshold_scale=[],
#                fill_color='BuGn',
#                legend_name='pm2.5',
#                line_opacity=0.5,
#                fill_opacity=0.8
#                )
folium.GeoJson(taichungmap_1x1,
               name='PM2.5',
               style_function=lambda x: {
                   'fillColor': map_color(x['properties'][variable]),
                   'color': 'black',
                   'weight': 0,
                   'fillOpacity': 0.7
               },
               highlight_function=lambda x: {
                   'weight': 3,
                   'color': 'black'
               },
               tooltip=folium.GeoJsonTooltip(fields=['Id', 'PM2.5'],
                                             aliases=['Id', 'PM2.5'],
                                             labels=True,
                                             sticky=True)).add_to(fmap)
f
# fmap.choropleth(
#               geo_data=taichung,
#               name='taichung',
#               line_opacity=0.5,
#               fill_opacity=0
#                )
#微型感測器logo
epa_micro_url= 'https://ci.taiwan.gov.tw/dsp/img/map_icon/air_quality.png'
# 環保署 logo
epa_icon_url = 'https://www.epa.gov.tw/public/MMO/epa/Epa_Logo_01_LOGO.gif'

station = folium.FeatureGroup(name="環保署", show=True)
for i in (range(15)):
    station.add_child(
        folium.Marker(
            location=[df1['Latitude'][i + 1], df1['Longitude'][i + 1]],
            popup=("<b>NAME:</b> {NAME}<br>"
                   " <p><b>PM2.5:</b> {PM25}<br>"
                   " <p><b>TIME:</b> {TIME}<br>").format(
                       NAME=str(df1['SiteName'][i + 1]),
                       PM25=str(df1['PM2.5'][i + 1]),
                       TIME=str(times[39:55])),
            icon=folium.CustomIcon(epa_icon_url,
                                   icon_size=(23,
                                              23))  # Creating a custom Icon
        ))




fmap.add_child(station)


fmap.add_child(map_color)
folium.LayerControl().add_to(fmap)
# lat/lon to map
# folium.LatLngPopup().add_to(fmap)
fmap.save('/var/www/html/pm25_original.html')  # 存成 final.html